#  Case Study: Jacobi Iteration - NVSHMEM

In this notebook and the next we will look at a Laplace equation solver using Jacobi iteration. This problem will give us excellent opportunity to explore the common motif of handling data communication at boundary points between distributed data.

## Objectives

By the time you complete this notebook you will:

- Be able to use NVSHMEM to handle boundary point communications in multi GPU algorithms with distributed data.

## Introduction to the Algorithm

By no means do you need to fully understand this algorithm to be able to accomplish the objectives of the course. However, for the curious, and for context, we will start by spending some time discussing the algorithm.

### The Laplace Equation

A common motif in finite-element/finite-volume/finite-difference applications is the solution of elliptic partial differential equations with relaxation methods. Perhaps the simplest elliptic PDE is the Laplace equation:

$$
\nabla^2\, f = 0
$$

where $\nabla^2 = \nabla \cdot \nabla$ is the Laplacian operator (sum of second derivatives for all coordinate directions) and $f = f(\mathbf{r})$ is a scalar field as a function of the spatial vector coordinate $\mathbf{r}$. The Laplace equation can be used to solve, for example, the equilibrium distribution of temperature on a metal plate that is heated to a fixed temperature on its edges.

In one dimension, where $f = f(x)$, this equation is:

$$
\frac{\partial^2}{\partial x^2} f = 0
$$

Suppose we want to solve this equation over a domain $x = [0, L]$, given fixed boundary conditions $f(0) = T_{\rm left}$ and $f(L) = T_{\rm right}$. That is, we want to know what the temperature distribution looks like in the interior of the domain as a function of $x$. A common approach is to discretize space into a set of $N$ points, located at $0, L\, /\, (N - 1),\, 2L\,/\,(N - 1),\, ...,\, (N - 2)\,L\, /\, (N - 1),\, L$. The leftmost and rightmost points will remain at the fixed temperatures $T_{\rm left}$ and $T_{\rm right}$ respectively, while the interior $N-2$ points are the unknowns we need to solve for. The distance between the points is $\Delta x = L\, /\, (N - 1)$, and we will store the points in an array of length $N$. For each index $i$ in the (zero-indexed) array, the coordinate position is $i\, L\, /\, (N - 1) = i\, \Delta x$.

In a discretized spatial domain, the derivatives of the field at index $i$ are some function of the nearby points. For example, a simple discretization of the first derivative would be:

$$
\frac{\partial}{\partial x} f_i = (f_{i+1} - f_{i-1})\ /\ (2\, \Delta x)
$$

<center><img src="images/1D_finite_differencing.png" width="700"></center>

While a simple discretization of the second derivative would be:

$$
\frac{\partial^2}{\partial x^2} f_i = (f_{i+1} - 2\, f_{i} + f_{i-1})\ /\ (\Delta x^2)
$$

If we set this expression equal to zero to satisfy the Laplace equation, we get:

$$
f_{i+1} - 2\, f_{i} + f_{i-1} = 0
$$

Solving this for $f_{i}$, we get:

$$
f_{i} = (f_{i+1} + f_{i-1})\ / \ 2
$$

### Jacobi Iteration to Solve

Although $f_{i+1}$ and $f_{i-1}$ are also varying (except at the boundary points $i == 0$ and $i == N-1$), it turns out that we can simply *iterate* on this solution for $f_{i}$ many times until the solution is sufficiently equilibrated. That is, if in every iteration we take the old solution to $f$, and then at every point in the new solution set it equal to the average of the two neighboring points from the old solution, we will eventually solve for the equilibrium distribution of $f$.

Depicting this approach in (serial) pseudo-code we get:

```
while (error > tolerance):
    l2_norm = 0
    for i = 1, N-2:
        f[i] = 0.5 * (f_old[i-1] + f_old[i+1])
        l2_norm += (f[i] - f_old[i]) * (f[i] - f_old[i])
    error = sqrt(l2_norm / N)
    swap(f_old, f)
```

### A Single GPU CUDA Implementation

This example is implemented in standard CUDA for a single GPU in the code `jacobi.cu`. Take some time to review the algorithm and its parallel implementation. As before, we are not aiming for the highest-performing solution, just something that sketches out the basic idea.

In [1]:
%%writefile jacobi.cu
#include <iostream>
#include <limits>
#include <cstdio>
#define NUM_POINTS 4194304
#define TOLERANCE  0.0001
#define MAX_ITERS  1000

__global__ void jacobi (const float* f_old, float* f, float* l2_norm, int N)
{
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    if (idx > 0 && idx < N - 1) 
    {
        f[idx] = 0.5f * (f_old[idx+1] + f_old[idx-1]);

        float l2 = (f[idx] - f_old[idx]) * (f[idx] - f_old[idx]);

        atomicAdd(l2_norm, l2);
    }
}

__global__ void initialize (float* f, float T_left, float T_right, int N)
{
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    
    if (idx == 0) 
    {
        f[idx] = T_left;
    }
    else if (idx == N - 1) 
    {
        f[idx] = T_right;
    }
    else if (idx < N - 1) 
    {
        f[idx] = 0.0f;
    }
}

int main(int argc, char **argv) 
{
    // Allocate space for the grid data, and the temporary buffer
    // for the "old" data.
    float* f_old;
    float* f;

    cudaMalloc(&f_old, NUM_POINTS * sizeof(float));
    cudaMalloc(&f, NUM_POINTS * sizeof(float));

    // Allocate memory for the L2 norm, on both the host and device.
    float* l2_norm = (float*) malloc(sizeof(float));
    float* d_l2_norm;
    cudaMalloc(&d_l2_norm, sizeof(float));

    // Initialize the error to a large number.
    float error = std::numeric_limits<float>::max();

    // Initialize the data.
    int threads_per_block = 256;
    int blocks = (NUM_POINTS + threads_per_block - 1) / threads_per_block;

    float T_left = 5.0f;
    float T_right = 10.0f;
    initialize<<<blocks, threads_per_block>>>(f_old, T_left, T_right, NUM_POINTS);
    initialize<<<blocks, threads_per_block>>>(f, T_left, T_right, NUM_POINTS);
    cudaDeviceSynchronize();

    // Now iterate until the error is sufficiently small.
    // As a safety mechanism, cap the number of iterations.
    int num_iters = 0;

    while (error > TOLERANCE && num_iters < MAX_ITERS) 
    {
        // Initialize the norm data to zero
        cudaMemset(d_l2_norm, 0, sizeof(float));

        // Launch kernel to do the calculation
        jacobi<<<blocks, threads_per_block>>>(f_old, f, d_l2_norm, NUM_POINTS);
        cudaDeviceSynchronize();

        // Swap f_old and f
        std::swap(f_old, f);

        // Update norm on host; calculate error by normalizing by number of zones and take square root
        cudaMemcpy(l2_norm, d_l2_norm, sizeof(float), cudaMemcpyDeviceToHost);

        if (*l2_norm == 0.0f) 
        {
            // Deal with first iteration
            error = 1.0f;
        }
        else 
        {
            error = std::sqrt(*l2_norm / NUM_POINTS);
        }

        if (num_iters % 10 == 0) 
        {
            std::cout << "Iteration = " << num_iters << " error = " << error << std::endl;
        }

        ++num_iters;
    }

    if (error <= TOLERANCE && num_iters < MAX_ITERS) 
    {
        std::cout << "Success!\n";
    }
    else 
    {
        std::cout << "Failure!\n";
    }

    // Clean up
    free(l2_norm);
    cudaFree(d_l2_norm);
    cudaFree(f_old);
    cudaFree(f);

    return 0;
}

Writing jacobi.cu


In [2]:
!nvcc -x cu -arch=sm_70 jacobi.cu -o jacobi 
!./jacobi

Iteration = 0 error = 0.00272958
Iteration = 10 error = 0.00034546
Iteration = 20 error = 0.000210903
Iteration = 30 error = 0.000157015
Iteration = 40 error = 0.000127122
Iteration = 50 error = 0.00010783
Success!


### Distributing with NVSHMEM

A very simple distribution strategy to multiple GPUs is to divide the domain into $M$ chunks (where $M$ is the number of GPUs). PE 0 will have points $[0, N\, /\, M - 1]$, PE 1 will have points $[N\, /\, M,\, 2\, N\, /\, M - 1]$, etc. In this approach, the communication between PEs needs to happen at the boundary points between PEs. For example, the update at point $i = N\, /\, M - 1$ on PE 0 is:

$f[N\, /\, M - 1] = (f[N\, /\, M] + f[N\, /\, M-2])\ /\ 2$

But this PE doesn't own the data point at $i = N\, /\, M$, it is owned by PE 1. So we will need to get that data point from the remote PE. To do so, we can use the [nvshmem_float_g()](https://docs.nvidia.com/hpc-sdk/nvshmem/api/gen/api/rma.html#nvshmem-get) API to get a scalar quantity on the remote PE.

```
float r = nvshmem_float_g(source, target_pe);
```

This then looks like the following. Note that with respect to PE 0, location `N / M` corresponds to index `0` of PE 1.

```
f_left = f[N / M - 2]
f_right = nvshmem_float_g(&f[0], 1)
f[N / M - 1] = (f_right + f_left) / 2
```

Let is to check the code `jacobi_nvshmem.cpp`:

In [4]:
%%writefile jacobi_nvshmem.cu
#include <iostream>
#include <limits>
#include <cstdio>
#include <nvshmem.h>
#include <nvshmemx.h>
#define NUM_POINTS 4194304
#define TOLERANCE  0.0001
#define MAX_ITERS  1000

__global__ void jacobi (const float* f_old, float* f, float* l2_norm, int N)
{
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    int my_pe = nvshmem_my_pe();
    int n_pes = nvshmem_n_pes();

    // Don't participate if we're the leftmost PE and on
    // the leftmost boundary point, or if we're the rightmost
    // PE and on the rightmost boundary point (as these are fixed).
    bool on_boundary = false;

    if (my_pe == 0 && idx == 0) 
    {
        on_boundary = true;
    }
    else if (my_pe == n_pes - 1 && idx == N - 1) 
    {
        on_boundary = true;
    }

    if (idx < N && !on_boundary) {
        // Retrieve the left and right points in the old data.
        // If we're fully in the interior of the local domain,
        // this is fully a local access. Otherwise, we need to
        // reach out to the remote PE to the left or right.

        float f_left;
        float f_right;

        if (idx == 0) 
        {
            // Note we don't get here if my_pe == 0.
            f_left = nvshmem_float_g(&f_old[N - 1], my_pe - 1);
        }
        else 
        {
            f_left = f_old[idx - 1];
        }

        if (idx == N - 1) 
        {
            // Note we don't get here if my_pe == n_pes - 1.
            f_right = nvshmem_float_g(&f_old[0], my_pe + 1);
        }
        else 
        {
            f_right = f_old[idx + 1];
        }

        f[idx] = 0.5f * (f_right + f_left);

        float l2 = (f[idx] - f_old[idx]) * (f[idx] - f_old[idx]);

        atomicAdd(l2_norm, l2);
    }
}

__global__ void initialize (float* f, float T_left, float T_right, int N)
{
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    int my_pe = nvshmem_my_pe();
    int n_pes = nvshmem_n_pes();

    if (idx == 0 && my_pe == 0) 
    {
        f[idx] = T_left;
    }
    else if (idx == N - 1 && my_pe == n_pes - 1) 
    {
        f[idx] = T_right;
    }
    else if (idx < N - 1) 
    {
        f[idx] = 0.0f;
    }
}

int main(int argc, char ** argv) 
{
    // Initialize NVSHMEM
    nvshmem_init();

    // Obtain our NVSHMEM processing element ID and number of PEs
    int my_pe = nvshmem_my_pe();
    int n_pes = nvshmem_n_pes();

    // Each PE (arbitrarily) chooses the GPU corresponding to its ID
    int device = my_pe;
    cudaSetDevice(device);

    // Each device handles a fraction 1 / n_pes of the work.
    const int N = NUM_POINTS / n_pes;

    // Allocate space for the grid data, and the temporary buffer
    // for the "old" data.
    float* f_old = (float*) nvshmem_malloc(N * sizeof(float));
    float* f = (float*) nvshmem_malloc(N * sizeof(float));

    // Allocate memory for the L2 norm, on both the host and device.
    float* l2_norm = (float*) malloc(sizeof(float));
    float* d_l2_norm = (float*) nvshmem_malloc(sizeof(float));

    // Initialize the error to a large number.
    float error = std::numeric_limits<float>::max();

    // Initialize the data.
    int threads_per_block = 256;
    int blocks = (N + threads_per_block - 1) / threads_per_block;

    float T_left = 5.0f;
    float T_right = 10.0f;
    initialize<<<blocks, threads_per_block>>>(f_old, T_left, T_right, N);
    initialize<<<blocks, threads_per_block>>>(f, T_left, T_right, N);
    cudaDeviceSynchronize();

    // Now iterate until the error is sufficiently small.
    // As a safety mechanism, cap the number of iterations.
    int num_iters = 0;

    while (error > TOLERANCE && num_iters < MAX_ITERS) 
    {
        // Initialize the norm data to zero
        cudaMemset(d_l2_norm, 0, sizeof(float));

        // Launch kernel to do the calculation
        jacobi<<<blocks, threads_per_block>>>(f_old, f, d_l2_norm, N);
        cudaDeviceSynchronize();

        // Swap f_old and f
        std::swap(f_old, f);

        // Sum the L2 norm over all PEs
        // Note this is a blocking API, so no explicit barrier is needed afterward
        nvshmem_float_sum_reduce(NVSHMEM_TEAM_WORLD, d_l2_norm, d_l2_norm, 1);

        // Update norm on host; calculate error by normalizing by number of zones and take square root
        cudaMemcpy(l2_norm, d_l2_norm, sizeof(float), cudaMemcpyDeviceToHost);

        if (*l2_norm == 0.0f) 
        {
            // Deal with first iteration
            error = 1.0f;
        }
        else 
        {
            error = std::sqrt(*l2_norm / NUM_POINTS);
        }

        if (num_iters % 10 == 0 && my_pe == 0) 
        {
            std::cout << "Iteration = " << num_iters << " error = " << error << std::endl;
        }

        ++num_iters;
    }

    if (my_pe == 0) 
    {
        if (error <= TOLERANCE && num_iters < MAX_ITERS) 
        {
            std::cout << "Success!\n";
        }
        else 
        {
            std::cout << "Failure!\n";
        }
    }

    free(l2_norm);
    nvshmem_free(d_l2_norm);
    nvshmem_free(f_old);
    nvshmem_free(f);

    // Finalize nvshmem
    nvshmem_finalize();

    return 0;
}

Writing jacobi_nvshmem.cpp


## Run the Code 

In [ ]:
!nvcc -rdc=true -ccbin g++ -gencode=arch=compute_70,code=sm_70 -I $NVSHMEM_HOME/include jacobi_nvshmem.cu -o jacobi_nvshmem -L $NVSHMEM_HOME/lib -lnvshmem -lnvidia-ml -lcuda -lcudart

In [ ]:
!nvshmrun -np 4 ./jacobi_nvshmem

## References

* NVIDIA DEVELOPER, https://developer.nvidia.com
(accessed January 12, 2023).